# Goal of the project

The goal of this project is to control a 2D quadrotor to reach a target while avoiding obstacles using a learned policy. You wil have to create your own Custom environment using the [stable_baselines3](https://stable-baselines3.readthedocs.io/en/master/) library and train a RL agent using PPO.

## 2D quadrotor

The quadrotor is depicted in the following figure
<img src='quadrotor.png' width="300">


The quadrotor model is written as
$$\begin{align} 
\dot{p_x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{p_y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $p_x$ is the horizontal and $p_y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $x = [p_x, v_x, p_y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines the problem and provides all the useful information about the robot and methods to simulate and animate it as shown below.

You can access the different parameters of the model in the following way:

In [1]:
import quadrotor

print("Mass    =", quadrotor.MASS)
print("Length  =", quadrotor.LENGTH)
print("Inertia =", quadrotor.INERTIA)
print("Dt      =", quadrotor.DT)
print("state size   =", quadrotor.DIM_STATE)
print("control size =", quadrotor.DIM_CONTROL)

Mass    = 0.5
Length  = 0.15
Inertia = 0.1
Dt      = 0.04
state size   = 6
control size = 2


The goal of this project is to learn a policy that can move the robot from any point to the red dot ($x^{\star} = [2, 0, 0, 0, 0, 0]^T$) while avoiding thee obstacles. The obstacles are represented by the black circles in the animation. You can check if the drone is in collision with an obstacle using the function ```quadrotor.check_collision```. 

## Create a RL environment
Using the [stable_baselines3](https://stable-baselines3.readthedocs.io/en/master/), create a [custom RL environment](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html) environment. You will have to follow the following steps:

1. Implement a step function than contrains the dynamics (you are free to use the ```quadrotor.next_state```) and a reward function. To speed-up the training, make sure to add a gravity compensation term in your dynamics (i.e. the drone should stay in place when the policy outputs zeros).
   The reward should be made of three terms:
   
- A positive term to incentivize the quadrotor to reach the target. You can start with a reward bounded between 0 and 1, e.g.
   $\operatorname{exp}(-\frac{1}{2} (x - x^{\star})Q(x - x^{\star}) -\frac{1}{2} (u - u_{\text{gravity}})R(u - u_{\text{gravity}}))$

   
- A large negative penality(e.g. -100)  if the robot get out of the following bounds:
$ p_x \in [-4, 4], \quad v_x \in [-10, 10] , \quad p_y \in [-4, 4] , \quad v_y \in [-10, 10] , \quad \theta \in [-2 \pi, 2 \pi] , \quad \omega \in [-10, 10] $.

 - A negative penalty if the robot hits the obstacle, e.g. -1. You should use the ```quadrotor.check_collision```.

Keep in mind that, in RL, the goal is to maximize a reward (and not minimize a cost like in Optimal Control).

2. Implement a reset function that initializes the state randomly. You can sample uniformly between $[-2, 2]$ for $p_x$ and $p_y$ and initialize the other terms to zero. Make sure to reject samples that are colliding with the obstacles using the ```quadrotor.check_collision```.

3. In the step function, stop the environment using ```truncated``` after 200 steps (Here is an [example](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/5_custom_gym_env.ipynb)).

4. In the step function, stop the environment if the drone goes outsite of the provided bounds using ```terminated``` (Here is an [example](https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/5_custom_gym_env.ipynb)).

5. Make sure that your environment is well defined using the ```check_env``` function.
   
## Training a policy with PPO   
Train a policy with PPO and use the learned policy to define a controller. Make sure that you can reach the target while avoiding the obstacles starting from $x_0 = [-2, 0, 0, 0 ,0, 0]$



Please submit your code (as runnable Jupyter Notebook), a pdf report and an mp4 video. In the report, explain your reward design and provide plots showing the trajectory of the quadrotor. The mp4 video should show the quadrotor animation starting from $x_0 = [-2, 0, 0, 0 ,0, 0]$. You can save your animation in the following way:


In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython


def controller(x, t):
    return np.zeros(2)

x_init = np.array([-2, 0, 0., 0 ,0, 0])
horizon_length = 200
# t, state, u = quadrotor.simulate(x_init, controller, horizon_length)
# quadrotor.animate_robot(state, u, save_mp4=True)

In [3]:
# # N = 100
# # u = np.array([[0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
# #              0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT]] * N).T

# def controller(x, t):
#     return np.array([0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
#                      0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT])
#     # return np.zeros(2)


# x_init = np.array([-2, 0, 0., 0, 0, 0])
# horizon_length = 100
# t, state, u = quadrotor.simulate(x_init, controller, horizon_length)
# quadrotor.animate_robot(state, u, save_mp4=False)

In [4]:
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO
from time import strftime
from stable_baselines3.common.env_util import make_vec_env

In [5]:
class QuadrotorEnv(gym.Env):

    def __init__(self, goal=np.array([2, 0, 0, 0, 0, 0])):
        super().__init__()
        self.u_gravity = np.array([0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
                                   0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT])
        self.action_space = spaces.Box(
            low=-10, high=10,
            shape=(quadrotor.DIM_CONTROL, ), dtype=np.float32)
        self.observation_space = spaces.Box(
            low=np.negative(np.array([4, 10, 4, 10, 2*np.pi, 10])),
            high=np.array([4, 10, 4, 10, 2*np.pi, 10]),
            shape=(quadrotor.DIM_STATE, ),
            dtype=np.float32)
        # self.observation_space = spaces.Box(
        #     low=-1e6,
        #     high=1e6,
        #     shape=(quadrotor.DIM_STATE, ),
        #     dtype=np.float32)
        self.Q = np.diag(np.array([31.25, 0.03125, 31.25, 0.03125,
                                   3.125, 0.03125], dtype=np.float32))
        self.R = np.diag(np.array([0.75, 0.75], dtype=np.float32))
        self.goal = goal
        self.current_state = np.zeros(6)
        self.max_steps = 200
        self.current_step = 0  # track steps per episode
        self.reset()

    def step(self, action):
        # if action not in self.action_space:
        #     raise ValueError(
        #         f'The chosen action {action} does not exist in the action space {self.action_space}')
        # if observation not in self.observation_space:
        #     raise ValueError(
        #         f'The current observation {observation} does not exist in the observation space {self.observation_space}')

        # action = action + self.u_gravity
        self.current_step += 1

        # Compute next state
        next_state = quadrotor.next_state(self.current_state, action)
        next_state = np.asarray(next_state, dtype=np.float32)
        terminated = False

        # Check bounds and set terminated if outside
        if next_state not in self.observation_space:
            terminated = True
            truncated = False
            reward = -100  # large negative penalty for going out of bounds
            return next_state, reward, terminated, truncated, {}

        # Check collision
        collided = quadrotor.check_collision(next_state)
        if collided:
            # Negative penalty for collision
            terminated = True
            truncated = False
            reward = -1
            return next_state, reward, terminated, truncated, {}

        # Check if goal reached
        # Instead of checking exact equality, check proximity:
        # if np.linalg.norm(next_state - self.goal) < 1e-6:  # threshold
        #     terminated = True
        #     truncated = False
        #     # You can give a high positive reward or rely on the reward function
        #     # defined in _get_reward
        # else:
        #     terminated = False

        # Check step count for truncation
        if self.current_step >= self.max_steps:
            truncated = True
        else:
            truncated = False

        # Compute reward
        reward = self._get_reward(action, next_state)

        self.current_state = next_state
        return next_state, reward, terminated, truncated, {}

    def reset(self, seed=42, options=None):
        super().reset(seed=seed, options=options)
        p_x = self.np_random.uniform(-2, 2)
        p_y = self.np_random.uniform(-2, 2)
        v_x = 0.0
        v_y = 0.0
        theta = 0.0
        omega = 0.0
        init_state = np.array(
            [p_x, v_x, p_y, v_y, theta, omega], dtype=np.float32)

        # Check collision
        while quadrotor.check_collision(init_state):
            p_x = self.np_random.uniform(-2, 2)
            p_y = self.np_random.uniform(-2, 2)
            init_state = np.array([p_x, 0, p_y, 0, 0, 0], dtype=np.float32)

        info = {}
        init_state = np.asarray(init_state, dtype=np.float32)
        self.current_state = init_state
        self.current_step = 0
        return init_state, info

    def render(self):
        ...

    def close(self):
        ...

    def _get_reward(self, action, observation) -> float:
        return 2.5 * np.exp(
            -((1/2) * ((observation - self.goal).T @
                       self.Q @ (observation - self.goal)))
            - ((1/2) * ((action - self.u_gravity).T @
                        self.R @ (action - self.u_gravity)))
        )

    # def _within_bounds(self, state):
    #     p_x, v_x, p_y, v_y, theta, omega = state
    #     if p_x < -4 or p_x > 4:
    #         return False
    #     if v_x < -10 or v_x > 10:
    #         return False
    #     if p_y < -4 or p_y > 4:
    #         return False
    #     if v_y < -10 or v_y > 10:
    #         return False
    #     if theta < -2*np.pi or theta > 2*np.pi:
    #         return False
    #     if omega < -10 or omega > 10:
    #         return False
    #     return True


check_env(QuadrotorEnv(), warn=True)

/home/shantanu/Documents/Academics/NYUMSMRFALL24/Reinforcement Learning and Optimal Control for Robotics (ROB-GY 6323)/Project 2/.venv/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/shantanu/Documents/Academics/NYUMSMRFALL24/Reinforcement Learning and Optimal Control for Robotics (ROB-GY 6323)/Project 2/.venv/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/home/shantanu/Documents/Academics/NYUMSMRFALL24/Reinforcement Learning and Optimal Control for Robotics (ROB-GY 6323)/Project 2/.venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:461: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  

In [6]:
# # N = 100
# env = QuadrotorEnv()
# quadrotor_model = PPO(policy='MlpPolicy', env=env, verbose=1,
#                       n_steps=horizon_length)
# quadrotor_model.learn(total_timesteps=1e4)
# quadrotor_model.save(
#     f'saved_models/quadrotor_model_{strftime("%Y%m%d_%H%M%S")}')

# env = QuadrotorEnv()
n_envs = 200
vec_env = make_vec_env(QuadrotorEnv, n_envs=n_envs, seed=42)
quadrotor_model = PPO(
    policy='MlpPolicy',
    env=vec_env,
    verbose=0,
    # n_steps=horizon_length*32,
    # batch_size=horizon_length*n_envs,
    device='cpu',
    seed=42,
    # clip_range=0.25
)
total_timesteps = 1e7
# consider a larger number of steps
quadrotor_model.learn(total_timesteps=total_timesteps,
                      progress_bar=True)
quadrotor_model.save(
    f'saved_models/quadrotor_model_{strftime("%Y%m%d_%H%M%S")}')

Output()

In [7]:

def controller(x, t):
    action = quadrotor_model.predict(observation=x)[0]
    # Add the same gravity compensation offset used in the environment
    u_gravity = np.array([0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT,
                          0.5 * quadrotor.MASS * quadrotor.GRAVITY_CONSTANT])
    return action + u_gravity


x_init = np.array([-2, 0, 0., 0, 0, 0])
horizon_length = 200
t, state, u = quadrotor.simulate(x_init, controller, horizon_length)
quadrotor.animate_robot(state, u, save_mp4=False)

FAILURE: The robot collided with an obstacle
FAILURE: The robot collided with an obstacle
FAILURE: The robot collided with an obstacle
FAILURE: The robot collided with an obstacle
FAILURE: The robot collided with an obstacle
FAILURE: The robot collided with an obstacle
FAILURE: The robot collided with an obstacle


Your browser does not support the video tag.